In [1]:
include("../../src/WaterLily.jl")
using LinearAlgebra: ⋅
using KernelAbstractions: get_backend, @index, @kernel
using Statistics

In [2]:
optPois = "DebugPois"
optSour = "Easy"
optBCon = "Neu"

computationID = optPois * "_" * optSour * "_" * optBCon * "_"
print(computationID,"\n")


DebugPois_Easy_Neu_


In [3]:

T=Float32
N = (2^4,2^4)
D = length(N)
Ng = N .+ 2
Nd = (Ng..., 2)

(18, 18, 2)

In [4]:
offset = -0.3

optSour=="Hard" && (SourceFunc(x,y) = (
    -4 *pi^2 * (1/N[1]^2 + 1/N[2]^2) * sin.(2*pi*(x/N[1].+offset)) .* sin.(2*pi*(y/N[2].+offset)) +
    -16*pi^2 * (1/N[1]^2 + 1/N[2]^2) * sin.(8*pi*(x/N[1].+offset)) .* sin.(8*pi*(y/N[2].+offset))
))

optSour=="Hard" && (ManufacSol(x,y) = (
    sin.(2*pi*(x/N[1].+offset)) .* sin.(2*pi*(y/N[2].+offset)) +
    sin.(8*pi*(x/N[1].+offset)) .* sin.(8*pi*(y/N[2].+offset))/4
))

optSour=="Easy" && (SourceFunc(x,y) = (
    4*pi^2 * (
        cos.(2*pi*(x/N[1])) .* (1 .-cos.(2*pi*(y/N[2])))/N[1]^2 +
        (1 .-cos.(2*pi*(x/N[1]))) .* cos.(2*pi*(y/N[2]))/N[2]^2
    )
))

optSour=="Easy" && (ManufacSol(x,y) = (
    (1 .-cos.(2*pi*(x/N[1]))) .* (1 .-cos.(2*pi*(y/N[2]))) 
))

ManufacSol (generic function with 1 method)

In [5]:
solIni = zeros(Ng)
source = zeros(Ng)
μ₀ = ones(Nd)
#WaterLily.BC!(μ₀,ntuple(zero, D))

# BCPer!(solIni)
# BCPer!(source)
# BCPer!(μ₀)

X = transpose(reshape([i-1.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))
Y = transpose(reshape([j-1.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))

source = SourceFunc(X,Y)
realSol = ManufacSol(X,Y)
originalSource = copy(source)

display(originalSource)

18×18 Matrix{Float64}:
 0.00581243  0.00581243  0.0279539  …  0.0279539  0.00581243  0.00581243
 0.00581243  0.00581243  0.0279539     0.0279539  0.00581243  0.00581243
 0.0279539   0.0279539   0.043219      0.043219   0.0279539   0.0279539
 0.068866    0.068866    0.0714251     0.0714251  0.068866    0.068866
 0.12232     0.12232     0.108278      0.108278   0.12232     0.12232
 0.180179    0.180179    0.148168   …  0.148168   0.180179    0.180179
 0.233633    0.233633    0.185021      0.185021   0.233633    0.233633
 0.274545    0.274545    0.213227      0.213227   0.274545    0.274545
 0.296686    0.296686    0.228492      0.228492   0.296686    0.296686
 0.296686    0.296686    0.228492      0.228492   0.296686    0.296686
 0.274545    0.274545    0.213227   …  0.213227   0.274545    0.274545
 0.233633    0.233633    0.185021      0.185021   0.233633    0.233633
 0.180179    0.180179    0.148168      0.148168   0.180179    0.180179
 0.12232     0.12232     0.108278      0.108278   

In [6]:
p = pTest = WaterLily.Poisson(solIni, μ₀, source)

Main.WaterLily.Poisson{Float64, Matrix{Float64}, Array{Float64, 3}}([1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0;;; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [0.0 0.0 … 0.0 0.0; 0.0 -4.0 … -4.0 0.0; … ; 0.0 -4.0 … -4.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 -0.25 … -0.25 0.0; … ; 0.0 -0.25 … -0.25 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.005812430292869291 0.005812430292869291 … 0.0058124302928693064 0.005812430292869291; 0.005812430292869291 0.005812430292869291 … 0.0058124302928693064 0.005812430292869291; … ; 0.0058124302928693064 0.0058124302928693064 … 0.005812430292869324 0.0058124302928693064; 0.005812430292869291 0.005812430292869291 … 0.0058124302928693064 0.0058124

In [7]:
res = WaterLily.solver!(p;log=true,tol=1e-12,itmx=1000)

display(p.x .- mean(p.x[2:end-1,2:end-1]))

18×18 Matrix{Float64}:
 -2.89699e-16  -2.89699e-16  -2.89699e-16  …  -2.89699e-16  -2.89699e-16
 -2.89699e-16  -1.01018      -1.00746         -1.01018      -2.89699e-16
 -2.89699e-16  -1.00746      -0.98215         -1.00746      -2.89699e-16
 -2.89699e-16  -1.00242      -0.935387        -1.00242      -2.89699e-16
 -2.89699e-16  -0.995849     -0.874289        -0.995849     -2.89699e-16
 -2.89699e-16  -0.98873      -0.808156     …  -0.98873      -2.89699e-16
 -2.89699e-16  -0.982154     -0.747057        -0.982154     -2.89699e-16
 -2.89699e-16  -0.977121     -0.700295        -0.977121     -2.89699e-16
 -2.89699e-16  -0.974397     -0.674987        -0.974397     -2.89699e-16
 -2.89699e-16  -0.974397     -0.674987        -0.974397     -2.89699e-16
 -2.89699e-16  -0.977121     -0.700295     …  -0.977121     -2.89699e-16
 -2.89699e-16  -0.982154     -0.747057        -0.982154     -2.89699e-16
 -2.89699e-16  -0.98873      -0.808156        -0.98873      -2.89699e-16
 -2.89699e-16  -0.995849    

In [ ]:
WaterLily.residual!(p)

x,r,ϵ,z = p.x,p.r,p.ϵ,p.z
for I ∈ WaterLily.inside(x)
    z[I] = ϵ[I] = r[I]*p.iD[I]
end
rho = r ⋅ z

In [ ]:
it=1000
for i in 1:it
    for I ∈ WaterLily.inside(x)
        z[I] = ϵ[I] = r[I]*p.iD[I]
    end
    rho = r ⋅ z
    WaterLily.BC!(ϵ)
    for I ∈ WaterLily.inside(x) 
        z[I] = WaterLily.mult(I,p.L,p.D,ϵ) 
    end
    alpha = rho/(z⋅ϵ)
    # print(rho, " ")
    # print(alpha, " ")
    for I ∈ WaterLily.inside(x) 
        x[I] += alpha*ϵ[I];
        r[I] -= alpha*z[I]
    end
    (i==it || abs(alpha)<1e-2) && return
    for I ∈ WaterLily.inside(x) 
        z[I] = r[I]*p.iD[I]
    end
    rho2 = r⋅z
    abs(rho2)<1e-8 && return
    beta = rho2/rho
    # print(beta, " \n")
    for I ∈ WaterLily.inside(x) 
        ϵ[I] = beta*ϵ[I]+z[I]
    end
    rho = rho2        
end

display(ϵ)

In [ ]:
WaterLily.inside(x)